# Detecting Payment Card Fraud

In this section, we'll look at a credit card fraud detection dataset, and build a binary classification model that can identify transactions as either fraudulent or valid, based on provided, *historical* data. In a [2016 study](https://nilsonreport.com/upload/content_promo/The_Nilson_Report_10-17-2016.pdf), it was estimated that credit card fraud was responsible for over 20 billion dollars in loss, worldwide. Accurately detecting cases of fraud is an ongoing area of research.

<img src=notebook_ims/fraud_detection.png width=50% />

### Labeled Data

The payment fraud data set (Dal Pozzolo et al. 2015) was downloaded from [Kaggle](https://www.kaggle.com/mlg-ulb/creditcardfraud/data). This has features and labels for thousands of credit card transactions, each of which is labeled as fraudulent or valid. In this notebook, we'd like to train a model based on the features of these transactions so that we can predict risky or fraudulent transactions in the future.

### Binary Classification

Since we have true labels to aim for, we'll take a **supervised learning** approach and train a binary classifier to sort data into one of our two transaction classes: fraudulent or valid.  We'll train a model on training data and see how well it generalizes on some test data.

The notebook will be broken down into a few steps:
* Loading and exploring the data
* Splitting the data into train/test sets
* Defining and training a LinearLearner, binary classifier
* Making improvements on the model
* Evaluating and comparing model test performance

### Making Improvements

A lot of this notebook will focus on making improvements, as discussed in [this SageMaker blog post](https://aws.amazon.com/blogs/machine-learning/train-faster-more-flexible-models-with-amazon-sagemaker-linear-learner/). Specifically, we'll address techniques for:

1. **Tuning a model's hyperparameters** and aiming for a specific metric, such as high recall or precision.
2. **Managing class imbalance**, which is when we have many more training examples in one class than another (in this case, many more valid transactions than fraudulent).

---

First, import the usual resources.

In [1]:
import io
import os
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 

import boto3
import sagemaker
from sagemaker import get_execution_role

%matplotlib inline

I'm storing my **SageMaker variables** in the next cell:
* sagemaker_session: The SageMaker session we'll use for training models.
* bucket: The name of the default S3 bucket that we'll use for data storage.
* role: The IAM role that defines our data and model permissions.

In [2]:
# sagemaker session, role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# S3 bucket name
bucket = sagemaker_session.default_bucket()


## Loading and Exploring the Data

Next, I am loading the data and unzipping the data in the file `creditcardfraud.zip`. This directory will hold one csv file of all the transaction data, `creditcard.csv`.

As in previous notebooks, it's important to look at the distribution of data since this will inform how we develop a fraud detection model. We'll want to know: How many data points we have to work with, the number and type of features, and finally, the distribution of data over the classes (valid or fraudulent).

In [3]:
# only have to run once
#!wget https://s3.amazonaws.com/video.udacity-data.com/topher/2019/January/5c534768_creditcardfraud/creditcardfraud.zip
#!unzip creditcardfraud


In [4]:
# read in the csv file
local_data = 'creditcard.csv'

# print out some data
transaction_df = pd.read_csv(local_data)
print('Data shape (rows, cols): ', transaction_df.shape)
print()
transaction_df.head()

Data shape (rows, cols):  (284807, 31)

Data shape (rows, cols):  (284807, 31)



,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


### EXERCISE: Calculate the percentage of fraudulent data

Take a look at the distribution of this transaction data over the classes, valid and fraudulent. 

Complete the function `fraudulent_percentage`, below. Count up the number of data points in each class and calculate the *percentage* of the data points that are fraudulent.

In [5]:
# Calculate the fraction of data points that are fraudulent
def fraudulent_percentage(transaction_df):
    '''Calculate the fraction of all data points that have a 'Class' label of 1; fraudulent.
       :param transaction_df: Dataframe of all transaction data points; has a column 'Class'
       :return: A fractional percentage of fraudulent data points/all points
    '''
    value_counts = transaction_df['Class'].value_counts()
    fraudulent_count = value_counts[1]
    return fraudulent_count / value_counts.sum()

Test out your code by calling your function and printing the result.

In [6]:
# call the function to calculate the fraud percentage
fraud_percentage = fraudulent_percentage(transaction_df)

print('Fraudulent percentage = ', fraud_percentage)
print('Total # of fraudulent pts: ', fraud_percentage*transaction_df.shape[0])
print('Out of (total) pts: ', transaction_df.shape[0])


Fraudulent percentage =  0.001727485630620034
Total # of fraudulent pts:  492.0
Out of (total) pts:  284807
Fraudulent percentage =  0.001727485630620034
Total # of fraudulent pts:  492.0
Out of (total) pts:  284807


### EXERCISE: Split into train/test datasets

In this example, we'll want to evaluate the performance of a fraud classifier; training it on some training data and testing it on *test data* that it did not see during the training process. So, we'll need to split the data into separate training and test sets.

Complete the `train_test_split` function, below. This function should:
* Shuffle the transaction data, randomly
* Split it into two sets according to the parameter `train_frac`
* Get train/test features and labels
* Return the tuples: (train_features, train_labels), (test_features, test_labels)

In [7]:
# split into train/test
def train_test_split(transaction_df, train_frac= 0.7, seed=1):
    '''Shuffle the data and randomly split into train and test sets;
       separate the class labels (the column in transaction_df) from the features.
       :param df: Dataframe of all credit card transaction data
       :param train_frac: The decimal fraction of data that should be training data
       :param seed: Random seed for shuffling and reproducibility, default = 1
       :return: Two tuples (in order): (train_features, train_labels), (test_features, test_labels)
       '''
    transaction_matrix = transaction_df.values
    
    np.random.seed(seed)
    np.random.shuffle(transaction_matrix)
    
    train_size = int(len(transaction_df.index)*train_frac)
    # shuffle and split the data
    train_labels = transaction_matrix[:train_size, -1]
    train_features = transaction_matrix[:train_size, :-1]
    test_labels = transaction_matrix[train_size:, -1]
    test_features = transaction_matrix[train_size:, :-1]
    
    return (train_features, train_labels), (test_features, test_labels)


### Test Cell

In the cells below, I'm creating the train/test data and checking to see that result makes sense. The tests below test that the above function splits the data into the expected number of points and that the labels are indeed, class labels (0, 1).

In [8]:
# get train/test data
(train_features, train_labels), (test_features, test_labels) = train_test_split(transaction_df, train_frac=0.7)

In [9]:
# manual test

# for a split of 0.7:0.3 there should be ~2.33x as many training as test pts
print('Training data pts: ', len(train_features))
print('Test data pts: ', len(test_features))
print()

# take a look at first item and see that it aligns with first row of data
print('First item: \n', train_features[0])
print('Label: ', train_labels[0])
print()

# test split
assert len(train_features) > 2.333*len(test_features), \
        'Unexpected number of train/test points for a train_frac=0.7'
# test labels
assert np.all(train_labels)== 0 or np.all(train_labels)== 1, \
        'Train labels should be 0s or 1s.'
assert np.all(test_labels)== 0 or np.all(test_labels)== 1, \
        'Test labels should be 0s or 1s.'
print('Tests passed!')

Training data pts:  199364
Test data pts:  85443

First item: 
 [ 1.19907000e+05 -6.11711999e-01 -7.69705324e-01 -1.49759145e-01
 -2.24876503e-01  2.02857736e+00 -2.01988711e+00  2.92491387e-01
 -5.23020325e-01  3.58468461e-01  7.00499612e-02 -8.54022784e-01
  5.47347360e-01  6.16448382e-01 -1.01785018e-01 -6.08491804e-01
 -2.88559430e-01 -6.06199260e-01 -9.00745518e-01 -2.01311157e-01
 -1.96039343e-01 -7.52077614e-02  4.55360454e-02  3.80739375e-01
  2.34403159e-02 -2.22068576e+00 -2.01145578e-01  6.65013699e-02
  2.21179560e-01  1.79000000e+00]
Label:  0.0

Tests passed!
Training data pts:  199364
Test data pts:  85443

First item: 
 [ 1.19907000e+05 -6.11711999e-01 -7.69705324e-01 -1.49759145e-01
 -2.24876503e-01  2.02857736e+00 -2.01988711e+00  2.92491387e-01
 -5.23020325e-01  3.58468461e-01  7.00499612e-02 -8.54022784e-01
  5.47347360e-01  6.16448382e-01 -1.01785018e-01 -6.08491804e-01
 -2.88559430e-01 -6.06199260e-01 -9.00745518e-01 -2.01311157e-01
 -1.96039343e-01 -7.52077614e-0

---
# Modeling

Now that you've uploaded your training data, it's time to define and train a model!

In this notebook, you'll define and train the SageMaker, built-in algorithm, [LinearLearner](https://sagemaker.readthedocs.io/en/stable/linear_learner.html). 

A LinearLearner has two main applications:
1. For regression tasks in which a linear line is fit to some data points, and you want to produce a predicted output value given some data point (example: predicting house prices given square area).
2. For binary classification, in which a line is separating two classes of data and effectively outputs labels; either 1 for data that falls above the line or 0 for points that fall on or below the line.

<img src='notebook_ims/linear_separator.png' width=50% />

In this case, we'll be using it for case 2, and we'll train it to separate data into our two classes: valid or fraudulent. 

### EXERCISE: Create a LinearLearner Estimator

You've had some practice instantiating built-in models in SageMaker. All estimators require some constructor arguments to be passed in. See if you can complete this task, instantiating a LinearLearner estimator, using only the [LinearLearner documentation](https://sagemaker.readthedocs.io/en/stable/linear_learner.html) as a resource. This takes in a lot of arguments, but not all are required. My suggestion is to start with a simple model, utilizing default values where applicable. Later, we will discuss some specific hyperparameters and their use cases.

#### Instance Types

It is suggested that you use instances that are available in the free tier of usage: `'ml.c4.xlarge'` for training and `'ml.t2.medium'` for deployment.

In [10]:
# import LinearLearner
from sagemaker import LinearLearner

# instantiate LinearLearner
linear = LinearLearner(
    role,
    instance_count=1,
    instance_type='ml.c4.xlarge',
    predictor_type='binary_classifier',
)

### EXERCISE: Convert data into a RecordSet format

Next, prepare the data for a built-in model by converting the train features and labels into numpy array's of float values. Then you can use the [record_set function](https://sagemaker.readthedocs.io/en/stable/linear_learner.html#sagemaker.LinearLearner.record_set) to format the data as a RecordSet and prepare it for training!

In [11]:
# create RecordSet of training data
np_train_x = train_features.astype('float32')
np_train_y = train_labels.astype('float32')
formatted_train_data = linear.record_set(np_train_x, labels=np_train_y)

### EXERCISE: Train the Estimator

After instantiating your estimator, train it with a call to `.fit()`, passing in the formatted training data.

In [12]:
%%time 
# train the estimator on formatted training data
linear.fit(formatted_train_data)

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2021-04-19 23:55:58 Starting - Starting the training job.2021-04-19 23:55:58 Starting - Starting the training job.....
2021-04-19 23:56:21 Starting - Launching requested ML instancesProfilerReport-1618876558: InProgress
.
2021-04-19 23:56:21 Starting - Launching requested ML instancesProfilerReport-1618876558: InProgress
...........
2021-04-19 23:57:21 Starting - Preparing the instances for training.
2021-04-19 23:57:21 Starting - Preparing the instances for training.......................
2021-04-19 23:59:22 Downloading - Downloading input data
2021-04-19 23:59:22 Training - Downloading the training image.
2021-04-19 23:59:22 Downloading - Downloading input data
2021-04-19 23:59:22 Training - Downloading the training image.Docker entrypoint called with argument(s): train
Running default environment configuration script
[04/19/2021 23:59:39 INFO 140176569943872] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_bat


2021-04-19 23:59:42 Training - Training image download completed. Training in progress.
2021-04-19 23:59:42 Training - Training image download completed. Training in progress.[2021-04-19 23:59:47.159] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 3, "duration": 6872, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1618876787.1591775, "EndTime": 1618876787.159253, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 0, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.12043031431801954, "count": 1, "min": 0.12043031431801954, "max": 0.12043031431801954}}}

#metrics {"StartTime": 1618876787.1593351, "EndTime": 1618876787.1593494, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 0, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.11774841896133806, "count": 1

[2021-04-19 23:59:47.159] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 3, "duration": 6872, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1618876787.1591775, "EndTime": 1618876787.159253, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 0, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.12043031431801954, "count": 1, "min": 0.12043031431801954, "max": 0.12043031431801954}}}

#metrics {"StartTime": 1618876787.1593351, "EndTime": 1618876787.1593494, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 0, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.11774841896133806, "count": 1, "min": 0.11774841896133806, "max": 0.11774841896133806}}}

#metrics {"StartTime": 1618876787.1593997, "EndTime": 1618876787.159418, "Dimensions": {"Algorithm": "Linear Learne

[2021-04-19 23:59:54.545] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 5, "duration": 7366, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1618876794.5455885, "EndTime": 1618876794.5456755, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.01697396406815879, "count": 1, "min": 0.01697396406815879, "max": 0.01697396406815879}}}

#metrics {"StartTime": 1618876794.545752, "EndTime": 1618876794.5457728, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.014988558927373071, "count": 1, "min": 0.014988558927373071, "max": 0.014988558927373071}}}

#metrics {"StartTime": 1618876794.5458283, "EndTime": 1618876794.5458457, "Dimensions": {"Algorithm": "Linear Le

...

[2021-04-20 00:01:13.083] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 27, "duration": 7284, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1618876873.0840423, "EndTime": 1618876873.0843039, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 12, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004969843113841723, "count": 1, "min": 0.004969843113841723, "max": 0.004969843113841723}}}

#metrics {"StartTime": 1618876873.084383, "EndTime": 1618876873.0844042, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 12, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.0043063429063888055, "count": 1, "min": 0.0043063429063888055, "max": 0.0043063429063888055}}}

#metrics {"StartTime": 1618876873.0844696, "EndTime": 1618876873.0844882, "Dimensions": {"Algorithm": "

[2021-04-20 00:01:13.083] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 27, "duration": 7284, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1618876873.0840423, "EndTime": 1618876873.0843039, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 12, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004969843113841723, "count": 1, "min": 0.004969843113841723, "max": 0.004969843113841723}}}

#metrics {"StartTime": 1618876873.084383, "EndTime": 1618876873.0844042, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 12, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.0043063429063888055, "count": 1, "min": 0.0043063429063888055, "max": 0.0043063429063888055}}}

#metrics {"StartTime": 1618876873.0844696, "EndTime": 1618876873.0844882, "Dimensions": {"Algorithm": "


2021-04-20 00:02:02 Completed - Training job completed
ProfilerReport-1618876558: NoIssuesFound
[2021-04-20 00:01:26.411] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 31, "duration": 6946, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1618876886.4119525, "EndTime": 1618876886.4120326, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 14, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004865119796901492, "count": 1, "min": 0.004865119796901492, "max": 0.004865119796901492}}}

#metrics {"StartTime": 1618876886.412106, "EndTime": 1618876886.41212, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 14, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004249175122035808, "count": 1, "min": 0.004249175122035808, "max": 0.004249175122035808}}}

#metrics {"


2021-04-20 00:02:02 Completed - Training job completed
ProfilerReport-1618876558: NoIssuesFound
[2021-04-20 00:01:26.411] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 31, "duration": 6946, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1618876886.4119525, "EndTime": 1618876886.4120326, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 14, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004865119796901492, "count": 1, "min": 0.004865119796901492, "max": 0.004865119796901492}}}

#metrics {"StartTime": 1618876886.412106, "EndTime": 1618876886.41212, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 14, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004249175122035808, "count": 1, "min": 0.004249175122035808, "max": 0.004249175122035808}}}

#metrics {"

Training seconds: 161
Billable seconds: 161
CPU times: user 912 ms, sys: 84.7 ms, total: 997 ms
Wall time: 6min 14s
Training seconds: 161
Billable seconds: 161
CPU times: user 912 ms, sys: 84.7 ms, total: 997 ms
Wall time: 6min 14s


### EXERCISE: Deploy the trained model

Deploy your model to create a predictor. We'll use this to make predictions on our test data and evaluate the model.

In [13]:
%%time 
# deploy and create a predictor
linear_predictor = linear.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


----------------------------------!!CPU times: user 303 ms, sys: 21 ms, total: 324 ms
Wall time: 8min 32s
CPU times: user 303 ms, sys: 21 ms, total: 324 ms
Wall time: 8min 32s


---
# Evaluating Your Model

Once your model is deployed, you can see how it performs when applied to the test data.

According to the deployed [predictor documentation](https://sagemaker.readthedocs.io/en/stable/linear_learner.html#sagemaker.LinearLearnerPredictor), this predictor expects an `ndarray` of input features and returns a list of Records.
> "The prediction is stored in the "predicted_label" key of the `Record.label` field."

Let's first test our model on just one test point, to see the resulting list.

In [14]:
# test one prediction
test_x_np = test_features.astype('float32')
result = linear_predictor.predict(test_x_np[0])

print(result)

[label {
  key: "predicted_label"
  value {
    float32_tensor {
      values: 0.0
    }
  }
}
label {
  key: "score"
  value {
    float32_tensor {
      values: 0.0018054395914077759
    }
  }
}
]
[label {
  key: "predicted_label"
  value {
    float32_tensor {
      values: 0.0
    }
  }
}
label {
  key: "score"
  value {
    float32_tensor {
      values: 0.0018054395914077759
    }
  }
}
]


### Helper function for evaluation


The provided function below, takes in a deployed predictor, some test features and labels, and returns a dictionary of metrics; calculating false negatives and positives as well as recall, precision, and accuracy.

In [15]:
# code to evaluate the endpoint on test data
# returns a variety of model metrics
def evaluate(predictor, test_features, test_labels, verbose=True):
    """
    Evaluate a model on a test set given the prediction endpoint.  
    Return binary classification metrics.
    :param predictor: A prediction endpoint
    :param test_features: Test features
    :param test_labels: Class labels for test data
    :param verbose: If True, prints a table of all performance metrics
    :return: A dictionary of performance metrics.
    """
    
    # We have a lot of test data, so we'll split it into batches of 100
    # split the test data set into batches and evaluate using prediction endpoint    
    prediction_batches = [predictor.predict(batch) for batch in np.array_split(test_features, 100)]
    
    # LinearLearner produces a `predicted_label` for each data point in a batch
    # get the 'predicted_label' for every point in a batch
    test_preds = np.concatenate([np.array([x.label['predicted_label'].float32_tensor.values[0] for x in batch]) 
                                 for batch in prediction_batches])
    
    # calculate true positives, false positives, true negatives, false negatives
    tp = np.logical_and(test_labels, test_preds).sum()
    fp = np.logical_and(1-test_labels, test_preds).sum()
    tn = np.logical_and(1-test_labels, 1-test_preds).sum()
    fn = np.logical_and(test_labels, 1-test_preds).sum()
    
    # calculate binary classification metrics
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    
    # printing a table of metrics
    if verbose:
        print(pd.crosstab(test_labels, test_preds, rownames=['actual (row)'], colnames=['prediction (col)']))
        print("\n{:<11} {:.3f}".format('Recall:', recall))
        print("{:<11} {:.3f}".format('Precision:', precision))
        print("{:<11} {:.3f}".format('Accuracy:', accuracy))
        print()
        
    return {'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn, 
            'Precision': precision, 'Recall': recall, 'Accuracy': accuracy}


### Test Results

The cell below runs the `evaluate` function. 

The code assumes that you have a defined `predictor` and `test_features` and `test_labels` from previously-run cells.

In [16]:
print('Metrics for simple, LinearLearner.\n')

# get metrics for linear predictor
metrics = evaluate(linear_predictor, 
                   test_features.astype('float32'), 
                   test_labels, 
                   verbose=True) # verbose means we'll print out the metrics


Metrics for simple, LinearLearner.

Metrics for simple, LinearLearner.

prediction (col)    0.0  1.0
actual (row)                
0.0               85269   33
1.0                  32  109

Recall:     0.773
Precision:  0.768
Accuracy:   0.999

prediction (col)    0.0  1.0
actual (row)                
0.0               85269   33
1.0                  32  109

Recall:     0.773
Precision:  0.768
Accuracy:   0.999



## Delete the Endpoint

I've added a convenience function to delete prediction endpoints after we're done with them. And if you're done evaluating the model, you should delete your model endpoint!

In [17]:
# Deletes a precictor.endpoint
def delete_endpoint(predictor):
        try:
            boto3.client('sagemaker').delete_endpoint(EndpointName=predictor.endpoint)
            print('Deleted {}'.format(predictor.endpoint))
        except:
            print('Already deleted: {}'.format(predictor.endpoint))

In [18]:
# delete the predictor endpoint 
delete_endpoint(linear_predictor)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Deleted linear-learner-2021-04-20-00-02-13-344
Deleted linear-learner-2021-04-20-00-02-13-344


---

# Model Improvements

The default LinearLearner got a high accuracy, but still classified fraudulent and valid data points incorrectly. Specifically classifying more than 30 points as false negatives (incorrectly labeled, fraudulent transactions), and a little over 30 points as false positives (incorrectly labeled, valid transactions). Let's think about what, during training, could cause this behavior and what we could improve.

**1. Model optimization**
* If we imagine that we are designing this model for use in a bank application, we know that users do *not* want any valid transactions to be categorized as fraudulent. That is, we want to have as few **false positives** (0s classified as 1s) as possible. 
* On the other hand, if our bank manager asks for an application that will catch almost *all* cases of fraud, even if it means a higher number of false positives, then we'd want as few **false negatives** as possible.
* To train according to specific product demands and goals, we do not want to optimize for accuracy only. Instead, we want to optimize for a metric that can help us decrease the number of false positives or negatives. 

<img src='notebook_ims/precision_recall.png' width=40% />
     
In this notebook, we'll look at different cases for tuning a model and make an optimization decision, accordingly.

**2. Imbalanced training data**
* At the start of this notebook, we saw that only about 0.17% of the training data was labeled as fraudulent. So, even if a model labels **all** of our data as valid, it will still have a high accuracy. 
* This may result in some overfitting towards valid data, which accounts for some **false negatives**; cases in which fraudulent data (1) is incorrectly characterized as valid (0).

So, let's address these issues in order; first, tuning our model and optimizing for a specific metric during training, and second, accounting for class imbalance in the training set. 


## Improvement: Model Tuning

Optimizing according to a specific metric is called **model tuning**, and SageMaker provides a number of ways to automatically tune a model.


### Create a LinearLearner and tune for higher precision 

**Scenario:**
* A bank has asked you to build a model that detects cases of fraud with an accuracy of about 85%. 

In this case, we want to build a model that has as many true positives and as few false negatives, as possible. This corresponds to a model with a high **recall**: true positives / (true positives + false negatives). 

To aim for a specific metric, LinearLearner offers the hyperparameter `binary_classifier_model_selection_criteria`, which is the model evaluation criteria for the training dataset. A reference to this parameter is in [LinearLearner's documentation](https://sagemaker.readthedocs.io/en/stable/linear_learner.html#sagemaker.LinearLearner). We'll also have to further specify the exact value we want to aim for; read more about the details of the parameters, [here](https://docs.aws.amazon.com/sagemaker/latest/dg/ll_hyperparameters.html).

I will assume that performance on a training set will be within about 5% of the performance on a test set. So, for a recall of about 85%, I'll aim for a bit higher, 90%.

In [22]:
# instantiate a LinearLearner
# tune the model for a higher recall
prefix = 'linear'
bucket_name= sagemaker_session.default_bucket()

output_path='s3://{}/{}/'.format(bucket_name, prefix)
linear_recall = LinearLearner(role=role,
                              train_instance_count=1, 
                              train_instance_type='ml.c4.xlarge',
                              predictor_type='binary_classifier',
                              output_path=output_path,
                              sagemaker_session=sagemaker_session,
                              epochs=15,
                              binary_classifier_model_selection_criteria='precision_at_target_recall', # target recall
                              target_recall=0.9) # 90% recall


train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


### Train the tuned estimator

Fit the new, tuned estimator on the formatted training data.

In [23]:
%%time 
# train the estimator on formatted training data
linear_recall.fit(formatted_train_data)

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2021-04-20 00:32:11 Starting - Starting the training job.2021-04-20 00:32:11 Starting - Starting the training job.....
2021-04-20 00:32:34 Starting - Launching requested ML instancesProfilerReport-1618878730: InProgress
.
2021-04-20 00:32:34 Starting - Launching requested ML instancesProfilerReport-1618878730: InProgress
.................
2021-04-20 00:33:54 Starting - Preparing the instances for training.
2021-04-20 00:33:54 Starting - Preparing the instances for training.................
2021-04-20 00:35:38 Downloading - Downloading input data.
2021-04-20 00:35:38 Downloading - Downloading input data.....
2021-04-20 00:36:12 Training - Training image download completed. Training in progress..
2021-04-20 00:36:12 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[04/20/2021 00:36:18 INFO 140443511715648] Reading default configuration from /opt/amazon/lib/python3.7/site-p

Docker entrypoint called with argument(s): train
Running default environment configuration script
[04/20/2021 00:36:18 INFO 140443511715648] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'init_bias': '0.0', 'optimizer': 'auto', 'loss': 'auto', 'margin': '1.0', 'quantile': '0.5', 'loss_insensitivity': '0.01', 'huber_delta': '1.0', 'num_classes': '1', 'accuracy_top_k': '3', 'wd': 'auto', 'l1': 'auto', 'momentum': 'auto', 'learning_rate': 'auto', 'beta_1': 'auto', 'beta_2': 'auto', 'bias_lr_mult': 'auto', 'bias_wd_mult': 'auto', 'use_lr_scheduler': 'true', 'lr_scheduler_step': 'auto'

[2021-04-20 00:36:33.293] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 5, "duration": 7318, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1618878993.2938004, "EndTime": 1618878993.2938893, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.01697396406815879, "count": 1, "min": 0.01697396406815879, "max": 0.01697396406815879}}}

#metrics {"StartTime": 1618878993.293967, "EndTime": 1618878993.2940872, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.014988558927373071, "count": 1, "min": 0.014988558927373071, "max": 0.014988558927373071}}}

#metrics {"StartTime": 1618878993.2942405, "EndTime": 1618878993.2942622, "Dimensions": {"Algorithm": "Linear Le

[2021-04-20 00:36:33.293] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 5, "duration": 7318, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1618878993.2938004, "EndTime": 1618878993.2938893, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.01697396406815879, "count": 1, "min": 0.01697396406815879, "max": 0.01697396406815879}}}

#metrics {"StartTime": 1618878993.293967, "EndTime": 1618878993.2940872, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.014988558927373071, "count": 1, "min": 0.014988558927373071, "max": 0.014988558927373071}}}

#metrics {"StartTime": 1618878993.2942405, "EndTime": 1618878993.2942622, "Dimensions": {"Algorithm": "Linear Le

...

[2021-04-20 00:37:50.201] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 27, "duration": 7384, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1618879070.2012575, "EndTime": 1618879070.2013736, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 12, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004969843113841723, "count": 1, "min": 0.004969843113841723, "max": 0.004969843113841723}}}

#metrics {"StartTime": 1618879070.2014692, "EndTime": 1618879070.201486, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 12, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.0043063429063888055, "count": 1, "min": 0.0043063429063888055, "max": 0.0043063429063888055}}}

#metrics {"StartTime": 1618879070.2015216, "EndTime": 1618879070.201531, "Dimensions": {"Algorithm": "L

[2021-04-20 00:37:50.201] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 27, "duration": 7384, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1618879070.2012575, "EndTime": 1618879070.2013736, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 12, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004969843113841723, "count": 1, "min": 0.004969843113841723, "max": 0.004969843113841723}}}

#metrics {"StartTime": 1618879070.2014692, "EndTime": 1618879070.201486, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 12, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.0043063429063888055, "count": 1, "min": 0.0043063429063888055, "max": 0.0043063429063888055}}}

#metrics {"StartTime": 1618879070.2015216, "EndTime": 1618879070.201531, "Dimensions": {"Algorithm": "L


2021-04-20 00:38:10 Uploading - Uploading generated training model[2021-04-20 00:38:04.696] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 31, "duration": 7395, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1618879084.6963632, "EndTime": 1618879084.6966057, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 14, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004865119796901492, "count": 1, "min": 0.004865119796901492, "max": 0.004865119796901492}}}

#metrics {"StartTime": 1618879084.6966827, "EndTime": 1618879084.6966965, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 14, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004249175122035808, "count": 1, "min": 0.004249175122035808, "max": 0.004249175122035808}}}

#metrics {"StartTime": 1618879084.6967


2021-04-20 00:38:10 Uploading - Uploading generated training model[2021-04-20 00:38:04.696] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 31, "duration": 7395, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1618879084.6963632, "EndTime": 1618879084.6966057, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 14, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004865119796901492, "count": 1, "min": 0.004865119796901492, "max": 0.004865119796901492}}}

#metrics {"StartTime": 1618879084.6966827, "EndTime": 1618879084.6966965, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 14, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004249175122035808, "count": 1, "min": 0.004249175122035808, "max": 0.004249175122035808}}}

#metrics {"StartTime": 1618879084.6967


2021-04-20 00:38:35 Completed - Training job completed
ProfilerReport-1618878730: NoIssuesFound

2021-04-20 00:38:35 Completed - Training job completed
ProfilerReport-1618878730: NoIssuesFound
Training seconds: 160
Billable seconds: 160
CPU times: user 1.05 s, sys: 94.4 ms, total: 1.15 s
Wall time: 6min 44s
Training seconds: 160
Billable seconds: 160
CPU times: user 1.05 s, sys: 94.4 ms, total: 1.15 s
Wall time: 6min 44s


### Deploy and evaluate the tuned estimator

Deploy the tuned predictor and evaluate it.

We hypothesized that a tuned model, optimized for a higher recall, would have fewer false negatives (fraudulent transactions incorrectly labeled as valid); did the number of false negatives get reduced after tuning the model?

In [24]:
%%time 
# deploy and create a predictor
recall_predictor = linear_recall.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


----------------------------------------------!!CPU times: user 389 ms, sys: 21.1 ms, total: 410 ms
Wall time: 11min 33s
CPU times: user 389 ms, sys: 21.1 ms, total: 410 ms
Wall time: 11min 33s


In [25]:
print('Metrics for tuned (recall), LinearLearner.\n')

# get metrics for tuned predictor
metrics = evaluate(recall_predictor, 
                   test_features.astype('float32'), 
                   test_labels, 
                   verbose=True)

Metrics for tuned (recall), LinearLearner.

Metrics for tuned (recall), LinearLearner.

prediction (col)    0.0   1.0
actual (row)                 
0.0               81913  3389
1.0                  10   131

Recall:     0.929
Precision:  0.037
Accuracy:   0.960

prediction (col)    0.0   1.0
actual (row)                 
0.0               81913  3389
1.0                  10   131

Recall:     0.929
Precision:  0.037
Accuracy:   0.960



## Delete the endpoint 

As always, when you're done evaluating a model, you should delete the endpoint. Below, I'm using the `delete_endpoint` helper function I defined earlier.

In [26]:
# delete the predictor endpoint 
delete_endpoint(recall_predictor)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Deleted linear-learner-2021-04-20-00-38-55-453
Deleted linear-learner-2021-04-20-00-38-55-453


---
## Improvement: Managing Class Imbalance

We have a model that is tuned to get a higher recall, which aims to reduce the number of false negatives. Earlier, we discussed how class imbalance may actually bias our model towards predicting that all transactions are valid, resulting in higher false negatives and true negatives. It stands to reason that this model could be further improved if we account for this imbalance.

To account for class imbalance during training of a binary classifier, LinearLearner offers the hyperparameter, `positive_example_weight_mult`, which is the weight assigned to positive (1, fraudulent) examples when training a binary classifier. The weight of negative examples (0, valid) is fixed at 1. 

### EXERCISE: Create a LinearLearner with a `positive_example_weight_mult` parameter

In **addition** to tuning a model for higher recall (you may use `linear_recall` as a starting point), you should *add* a parameter that helps account for class imbalance. From the [hyperparameter documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/ll_hyperparameters.html) on `positive_example_weight_mult`, it reads:
> "If you want the algorithm to choose a weight so that errors in classifying negative vs. positive examples have equal impact on training loss, specify `balanced`."

You could also put in a specific float value, in which case you'd want to weight positive examples more heavily than negative examples, since there are fewer of them.

In [27]:
# instantiate a LinearLearner

# include params for tuning for higher recall
# *and* account for class imbalance in training data
linear_balanced = LinearLearner(role=role,
                              train_instance_count=1, 
                              train_instance_type='ml.c4.xlarge',
                              predictor_type='binary_classifier',
                              output_path=output_path,
                              sagemaker_session=sagemaker_session,
                              epochs=15,
                              binary_classifier_model_selection_criteria='precision_at_target_recall', # target recall
                              positive_example_weight_mult='balanced',
                              target_recall=0.9) # 90% recall


train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


### EXERCISE: Train the balanced estimator

Fit the new, balanced estimator on the formatted training data.

In [29]:
%%time 
# train the estimator on formatted training data
linear_balanced.fit(formatted_train_data)

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2021-04-20 00:55:06 Starting - Starting the training job.2021-04-20 00:55:06 Starting - Starting the training job.....
2021-04-20 00:55:29 Starting - Launching requested ML instancesProfilerReport-1618880106: InProgress
.
2021-04-20 00:55:29 Starting - Launching requested ML instancesProfilerReport-1618880106: InProgress
...........
2021-04-20 00:56:29 Starting - Preparing the instances for training.
2021-04-20 00:56:29 Starting - Preparing the instances for training.......................
2021-04-20 00:58:37 Downloading - Downloading input data
2021-04-20 00:58:37 Training - Training image download completed. Training in progress..
2021-04-20 00:58:37 Downloading - Downloading input data
2021-04-20 00:58:37 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[04/20/2021 00:58:43 INFO 140713350600512] Reading default configuration from /opt/amazon/lib/python3.7/site-package

#metrics {"StartTime": 1618880323.7496645, "EndTime": 1618880323.7496998, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "Meta": "init_train_data_iter"}, "Metrics": {"Total Records Seen": {"sum": 12000.0, "count": 1, "min": 12000, "max": 12000}, "Total Batches Seen": {"sum": 12.0, "count": 1, "min": 12, "max": 12}, "Max Records Seen Between Resets": {"sum": 11000.0, "count": 1, "min": 11000, "max": 11000}, "Max Batches Seen Between Resets": {"sum": 11.0, "count": 1, "min": 11, "max": 11}, "Reset Count": {"sum": 2.0, "count": 1, "min": 2, "max": 2}, "Number of Records Since Last Reset": {"sum": 0.0, "count": 1, "min": 0, "max": 0}, "Number of Batches Since Last Reset": {"sum": 0.0, "count": 1, "min": 0, "max": 0}}}

[2021-04-20 00:58:50.931] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 3, "duration": 7181, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1618880330.931476, "EndTime": 1618

#metrics {"StartTime": 1618880323.7496645, "EndTime": 1618880323.7496998, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "Meta": "init_train_data_iter"}, "Metrics": {"Total Records Seen": {"sum": 12000.0, "count": 1, "min": 12000, "max": 12000}, "Total Batches Seen": {"sum": 12.0, "count": 1, "min": 12, "max": 12}, "Max Records Seen Between Resets": {"sum": 11000.0, "count": 1, "min": 11000, "max": 11000}, "Max Batches Seen Between Resets": {"sum": 11.0, "count": 1, "min": 11, "max": 11}, "Reset Count": {"sum": 2.0, "count": 1, "min": 2, "max": 2}, "Number of Records Since Last Reset": {"sum": 0.0, "count": 1, "min": 0, "max": 0}, "Number of Batches Since Last Reset": {"sum": 0.0, "count": 1, "min": 0, "max": 0}}}

[2021-04-20 00:58:50.931] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 3, "duration": 7181, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1618880330.931476, "EndTime": 1618

[2021-04-20 00:58:58.364] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 5, "duration": 7410, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1618880338.3641856, "EndTime": 1618880338.3642664, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.44001085551659663, "count": 1, "min": 0.44001085551659663, "max": 0.44001085551659663}}}

#metrics {"StartTime": 1618880338.3643484, "EndTime": 1618880338.3643641, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.4105252083629819, "count": 1, "min": 0.4105252083629819, "max": 0.4105252083629819}}}

#metrics {"StartTime": 1618880338.3644187, "EndTime": 1618880338.3644335, "Dimensions": {"Algorithm

...

[2021-04-20 01:00:17.594] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 27, "duration": 7027, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1618880417.5948575, "EndTime": 1618880417.5949452, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 12, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3411415676327806, "count": 1, "min": 0.3411415676327806, "max": 0.3411415676327806}}}

#metrics {"StartTime": 1618880417.5950432, "EndTime": 1618880417.5950658, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 12, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3368815051898285, "count": 1, "min": 0.3368815051898285, "max": 0.3368815051898285}}}

#metrics {"StartTime": 1618880417.5951219, "EndTime": 1618880417.595138, "Dimensions": {"Algorithm"

[2021-04-20 01:00:17.594] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 27, "duration": 7027, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1618880417.5948575, "EndTime": 1618880417.5949452, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 12, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3411415676327806, "count": 1, "min": 0.3411415676327806, "max": 0.3411415676327806}}}

#metrics {"StartTime": 1618880417.5950432, "EndTime": 1618880417.5950658, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 12, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3368815051898285, "count": 1, "min": 0.3368815051898285, "max": 0.3368815051898285}}}

#metrics {"StartTime": 1618880417.5951219, "EndTime": 1618880417.595138, "Dimensions": {"Algorithm"


2021-04-20 01:00:37 Uploading - Uploading generated training model[2021-04-20 01:00:24.569] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 29, "duration": 6960, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1618880424.569613, "EndTime": 1618880424.5697024, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 13, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3403380544844584, "count": 1, "min": 0.3403380544844584, "max": 0.3403380544844584}}}

#metrics {"StartTime": 1618880424.5697825, "EndTime": 1618880424.569804, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 13, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3361356634782187, "count": 1, "min": 0.3361356634782187, "max": 0.3361356634782187}}}

#metrics {"StartTime": 1618880424.


2021-04-20 01:00:37 Uploading - Uploading generated training model[2021-04-20 01:00:24.569] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 29, "duration": 6960, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1618880424.569613, "EndTime": 1618880424.5697024, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 13, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3403380544844584, "count": 1, "min": 0.3403380544844584, "max": 0.3403380544844584}}}

#metrics {"StartTime": 1618880424.5697825, "EndTime": 1618880424.569804, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 13, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3361356634782187, "count": 1, "min": 0.3361356634782187, "max": 0.3361356634782187}}}

#metrics {"StartTime": 1618880424.


2021-04-20 01:00:50 Completed - Training job completed
ProfilerReport-1618880106: NoIssuesFound

2021-04-20 01:00:50 Completed - Training job completed
ProfilerReport-1618880106: NoIssuesFound
Training seconds: 156
Billable seconds: 156
CPU times: user 932 ms, sys: 81.9 ms, total: 1.01 s
Wall time: 6min 14s
Training seconds: 156
Billable seconds: 156
CPU times: user 932 ms, sys: 81.9 ms, total: 1.01 s
Wall time: 6min 14s


### EXERCISE: Deploy and evaluate the balanced estimator

Deploy the balanced predictor and evaluate it. Do the results match with your expectations?

In [30]:
%%time 
# deploy and create a predictor
balanced_predictor = linear_balanced.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


------------------------------------------!!CPU times: user 359 ms, sys: 22.4 ms, total: 381 ms
Wall time: 10min 32s
CPU times: user 359 ms, sys: 22.4 ms, total: 381 ms
Wall time: 10min 32s


In [31]:
print('Metrics for balanced, LinearLearner.\n')

# get metrics for balanced predictor
metrics = evaluate(balanced_predictor, 
                   test_features.astype('float32'), 
                   test_labels, 
                   verbose=True)

Metrics for balanced, LinearLearner.

Metrics for balanced, LinearLearner.

prediction (col)    0.0   1.0
actual (row)                 
0.0               84277  1025
1.0                  12   129

Recall:     0.915
Precision:  0.112
Accuracy:   0.988

prediction (col)    0.0   1.0
actual (row)                 
0.0               84277  1025
1.0                  12   129

Recall:     0.915
Precision:  0.112
Accuracy:   0.988



## Delete the endpoint 

When you're done evaluating a model, you should delete the endpoint.

In [32]:
# delete the predictor endpoint 
delete_endpoint(balanced_predictor)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Deleted linear-learner-2021-04-20-01-01-20-487
Deleted linear-learner-2021-04-20-01-01-20-487


A note on metric variability: 

The above model is tuned for the best possible precision with recall fixed at about 90%. The recall is fixed at 90% during training, but may vary when we apply our trained model to a test set of data.

---
## Model Design

Now that you've seen how to tune and balance a LinearLearner. Create, train and deploy your own model. This exercise is meant to be more open-ended, so that you get practice with the steps involved in designing a model and deploying it.

### EXERCISE: Train and deploy a LinearLearner with appropriate hyperparameters, according to the given scenario

**Scenario:**
* A bank has asked you to build a model that optimizes for a good user experience; users should only ever have up to about 15% of their valid transactions flagged as fraudulent.

This requires that you make a design decision: Given the above scenario, what metric (and value) should you aim for during training?

You may assume that performance on a training set will be within about 5-10% of the performance on a test set. For example, if you get 80% on a training set, you can assume that you'll get between about 70-90% accuracy on a test set.

Your final model should account for class imbalance and be appropriately tuned. 

In [ ]:
%%time
# instantiate and train a LinearLearner

# include params for tuning for higher precision
# *and* account for class imbalance in training data


In [ ]:
%%time 
# deploy and evaluate a predictor


In [ ]:
## IMPORTANT
# delete the predictor endpoint after evaluation 


## Final Cleanup!

* Double check that you have deleted all your endpoints.
* I'd also suggest manually deleting your S3 bucket, models, and endpoint configurations directly from your AWS console.

You can find thorough cleanup instructions, [in the documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html).

---
# Conclusion

In this notebook, you saw how to train and deploy a LinearLearner in SageMaker. This model is well-suited for a binary classification task that involves specific design decisions and managing class imbalance in the training set.

Following the steps of a machine learning workflow, you loaded in some credit card transaction data, explored that data and prepared it for model training. Then trained, deployed, and evaluated several models, according to different design considerations!